In [1]:
import geopandas as gpd
import pandas as pd
import sys
import os
import gtfs_kit as gtk
import numpy as np
sys.path.insert(0, r'../../../')
from quetzal.io.gtfs_reader import importer
from quetzal.io.gtfs_reader.frequencies import hhmmss_to_seconds_since_midnight 
from quetzal.model import stepmodel


import warnings
warnings.filterwarnings("ignore")

In [2]:
time_range = ['6:00:00', '8:59:00'] # PPAM
#dates = ['20191015'] # the dates must be within the feed start and end dates
#https://pandas.pydata.org/docs/reference/api/pandas.DatetimeIndex.weekday.html
day = 'tuesday'
day_dict = {
    'monday': 0,
    'tuesday': 1,
    'wednesday': 2,
    'thursday': 3,
    'friday': 4,
    'saturday': 5,
    'sunday': 6
}
selected_day = day_dict[day]
selected_day

1

In [3]:
# GTFS are read recursively between each ancestor
files=[]
gtfs_folder = 'gtfs/paris/'
if os.path.exists(gtfs_folder):
    for filename in filter(lambda x: x[-4:] == '.zip', os.listdir(gtfs_folder)):
        files.append(gtfs_folder+filename)

#files = files[0:1]
files

['gtfs/paris/ratp.zip']

In [4]:
files = [
    "https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-unknown-via-rail-canada-gtfs-735.zip?alt=media",
    "https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-quebec-exo-sorel-varennes-gtfs-741.zip?alt=media",
    "https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-quebec-exo-sud-ouest-gtfs-742.zip?alt=media",
    "https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-quebec-exo-la-presquile-gtfs-743.zip?alt=media"
]

In [5]:
feeds=[]
for file in files:
    print('Importing {f}.zip'.format(f=file))
    feeds.append(importer.GtfsImporter(path=file, dist_units='m'))


Importing https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-unknown-via-rail-canada-gtfs-735.zip?alt=media.zip
Importing https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-quebec-exo-sorel-varennes-gtfs-741.zip?alt=media.zip
Importing https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-quebec-exo-sud-ouest-gtfs-742.zip?alt=media.zip
Importing https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-quebec-exo-la-presquile-gtfs-743.zip?alt=media.zip


In [6]:
for i in range(len(feeds)):
    print(i)
    if 'agency_id' not in feeds[i].routes:
        print(f'add agency_id to routes in {files[i]}')
        feeds[i].routes['agency_id'] = feeds[i].agency['agency_id'].values[0]

    
    if 'pickup_type' not in feeds[i].stop_times:
        print(f'picjup_type missing in stop_times. set to 0 in {files[i]}')
        feeds[i].stop_times['pickup_type'] = 0
    
    if 'drop_off_type' not in feeds[i].stop_times:
        print(f'drop_odd_type missing in stop_times. set to 0 in {files[i]}')
        feeds[i].stop_times['drop_off_type'] = 0
        
    if 'parent_station' not in feeds[i].stops:
        print(f'parent_station missing in stops. set to NaN in {files[i]}')
        feeds[i].stops['parent_station'] = np.nan
    feeds[i].stop_times['pickup_type'].fillna(0, inplace=True)
    feeds[i].stop_times['drop_off_type'].fillna(0, inplace=True)
    
    
    
    if 'shape_dist_traveled' not in feeds[i].stop_times.columns:
        feeds[i] = gtk.append_dist_to_stop_times(feeds[i])
    elif  any(feeds[i].stop_times['shape_dist_traveled'].isnull()):
        feeds[i] = gtk.append_dist_to_stop_times(feeds[i])
    feeds[i].stop_times.loc[(feeds[i].stop_times['stop_sequence'] == 1), 'shape_dist_traveled'] = feeds[i].stop_times[feeds[i].stop_times['stop_sequence'] == 1]['shape_dist_traveled'].fillna(0.0)
    '''
    if feeds[i].stop_times['shape_dist_traveled'].max() < 100:
        print(f'convert to meters : {files[i]}')
        feeds[i].dist_units = 'km'
        feeds[i] = gtk.convert_dist(feeds[i], new_dist_units='m')
    '''
   

    assert all(~feeds[i].routes['agency_id'].isna())
    
    feeds[i].stop_times['arrival_time'] = feeds[i].stop_times['departure_time']

0
1
parent_station missing in stops. set to NaN in https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-quebec-exo-sorel-varennes-gtfs-741.zip?alt=media
2
parent_station missing in stops. set to NaN in https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-quebec-exo-sud-ouest-gtfs-742.zip?alt=media
3
parent_station missing in stops. set to NaN in https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ca-quebec-exo-la-presquile-gtfs-743.zip?alt=media


In [7]:

dates =[]
for feed in feeds:
    min_date = feed.calendar['start_date'].unique().min()
    max_date = feed.calendar['end_date'].unique().max()
    # get date range 
    s = pd.date_range(min_date, max_date, freq='D').to_series()
    # get dayofweek selected and take first one
    s = s[s.dt.dayofweek==selected_day][0]
    # format  ex: ['20231011'] and append
    dates.append([f'{s.year}{str(s.month).zfill(2)}{str(s.day).zfill(2)}'])


In [8]:
dates

[['20230103'], ['20231003'], ['20231017'], ['20231017']]

In [9]:
feeds_t = []

for i, feed in enumerate(feeds):
    feed_t = feed.restrict(dates=dates[i], time_range=time_range)
    if len(feed_t.trips) > 0:
        feeds_t.append(feed_t)


In [10]:
for i in range(len(feeds_t)):
    if 'shape_dist_traveled' not in feeds_t[i].stop_times.columns:
        feeds_t[i] = gtk.append_dist_to_stop_times(feeds_t[i])
    feeds_t[i].stop_times.loc[(feeds_t[i].stop_times['stop_sequence'] == 1), 'shape_dist_traveled'] = feeds_t[i].stop_times[feeds_t[i].stop_times['stop_sequence'] == 1]['shape_dist_traveled'].fillna(0.0)

    if feeds_t[i].stop_times['shape_dist_traveled'].max() < 100:
        print(f'convert to meters')
        feeds_t[i].dist_units = 'km'
        feeds_t[i] = gtk.convert_dist(feeds_t[i], new_dist_units='m')


convert to meters
convert to meters
convert to meters


In [12]:
%%time
feeds_frequencies = []

for i in range(len(feeds_t)):
    print(i)
    feed_s = feeds_t[i].copy()
    feed_s.group_services()

    feed_s.build_stop_clusters(distance_threshold=50)
    feed_s.build_patterns(on='cluster_id')

    feed_frequencies = feed_s.convert_to_frequencies(time_range=time_range)
    shapes = feed_frequencies.shapes is not None
    feed_frequencies.build_links_and_nodes(log=False, 
                                           shape_dist_traveled=True, 
                                           from_shape=shapes, 
                                           stick_nodes_on_links=shapes,
                                           keep_origin_columns=['departure_time','pickup_type'],
                                           keep_destination_columns=['arrival_time','drop_off_type'])
    feeds_frequencies.append(feed_frequencies)

0
1
2
3
CPU times: user 27.6 s, sys: 32.9 ms, total: 27.6 s
Wall time: 27.7 s


In [13]:
mapping = {0:'tram', 1:'subway', 2:'rail', 3:'bus',4:'ferry',5:'cable_car',6:'gondola',7:'funicular', 700:'bus', 1501:'taxi'}
retire = ['taxi']
for feed_frequencies in feeds_frequencies:
    feed_frequencies.links['route_type'] = feed_frequencies.links['route_type'].apply(
        lambda t: mapping.get(t, np.nan)
    )
    
    assert not any(feed_frequencies.links['route_type'].isna())
    feed_frequencies.links = feed_frequencies.links[~feed_frequencies.links['route_type'].isin(retire)]

for feed_frequencies in feeds_frequencies:
    feed_frequencies.links.loc[feed_frequencies.links['time'] == 0,'time'] = 1.0

# create Model

In [14]:
columns=['trip_id','route_id','agency_id','direction_id','a','b', 'shape_dist_traveled',
                                    'link_sequence','time','headway','pickup_type', 'drop_off_type',
                                    'route_short_name','route_type','route_color','geometry']

In [15]:
sm = stepmodel.StepModel(epsg=4326, coordinates_unit='meter')

links_concat = []; nodes_concat = []
for feed_frequencies in feeds_frequencies:
    links_concat.append(feed_frequencies.links)
    nodes_concat.append(feed_frequencies.nodes)

sm.links = pd.concat(links_concat)

for col in columns:
    if col not in sm.links.columns:
        sm.links[col] = np.nan
        
sm.links = sm.links[columns]
sm.nodes = pd.concat(nodes_concat)[['stop_id','stop_name','stop_code','geometry']]

sm.nodes = sm.nodes.reset_index(drop=True).sort_index()
sm.links = sm.links.reset_index(drop=True).sort_index()


sm.nodes.loc[sm.nodes['stop_code'].isna(),'stop_code'] = sm.nodes.loc[sm.nodes['stop_code'].isna(),'stop_id'] 
sm.nodes.drop_duplicates(subset=['stop_id'], inplace=True)

sm.links['trip_id'] = sm.links['agency_id'] +'_' +sm.links['trip_id']
sm.links['route_id'] = sm.links['agency_id'] +'_' +sm.links['route_id']

sm.links = sm.links.sort_values(['route_type','trip_id']).reset_index(drop=True)

dnodes = ('node_' +sm.nodes.reset_index().set_index('stop_id')['index'].astype(str)).to_dict()
sm.nodes.index = 'node_' +sm.nodes.index.astype(str)

sm.links.index = 'link_' +sm.links.index.astype(str)

sm.links['a'] = sm.links['a'].apply(lambda a: dnodes.get(a))
sm.links['b'] = sm.links['b'].apply(lambda a: dnodes.get(a))

sm.links.drop_duplicates(subset=['trip_id','link_sequence'], inplace=True)

# Tag route with only one trip
time_slot = np.diff([hhmmss_to_seconds_since_midnight(time) for time in time_range])[0]
sm.links.loc[(time_slot/sm.links['headway']) < 2.0, 'headway'] = np.nan

sm.links = sm.links.to_crs(4326)
sm.nodes = sm.nodes.to_crs(4326)

# test

In [108]:
df = pd.read_csv('sources.csv')

In [112]:
df[df['location.subdivision_name']=='Québec']

,mdb_source_id,data_type,entity_type,location.country_code,location.subdivision_name,location.municipality,provider,name,note,static_reference,...,urls.api_key_parameter_name,urls.latest,urls.license,location.bounding_box.minimum_latitude,location.bounding_box.maximum_latitude,location.bounding_box.minimum_longitude,location.bounding_box.maximum_longitude,location.bounding_box.extracted_on,status,features
738,739,gtfs,NaN,CA,Québec,La Pêche,Régie intermunicipale de transport des Collines,NaN,NaN,NaN,...,NaN,https://storage.googleapis.com/storage/v1/b/md...,NaN,45.407263,45.919647,-77.076473,-75.617139,2022-03-17T13:15:38+00:00,NaN,NaN
739,740,gtfs,NaN,CA,Québec,Gatineau,Société de transport de l'Outaouais,NaN,NaN,NaN,...,NaN,https://storage.googleapis.com/storage/v1/b/md...,NaN,45.381261,45.592886,-75.890201,-75.390856,2022-03-17T13:15:42+00:00,NaN,NaN
740,741,gtfs,NaN,CA,Québec,Varennes,Exo Sorel-Varennes,NaN,NaN,NaN,...,NaN,https://storage.googleapis.com/storage/v1/b/md...,NaN,45.524297,46.046225,-73.540802,-73.084444,2022-03-17T13:15:44+00:00,NaN,NaN
741,742,gtfs,NaN,CA,Québec,NaN,Exo Sud-ouest,NaN,NaN,NaN,...,NaN,https://storage.googleapis.com/storage/v1/b/md...,NaN,45.251963,45.498100,-74.132096,-73.564442,2022-03-17T13:15:45+00:00,NaN,NaN
742,743,gtfs,NaN,CA,Québec,NaN,Exo La Presqu'île,NaN,NaN,NaN,...,NaN,https://storage.googleapis.com/storage/v1/b/md...,NaN,45.350844,45.513436,-74.315165,-73.682942,2022-03-17T13:15:47+00:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,1369,gtfs-rt,tu,CA,Québec,NaN,Exo Le Richelain,NaN,NaN,746.0,...,token,NaN,https://exo.quebec/fr/a-propos/donnees-ouverte...,45.351194,45.548627,-73.565996,-73.387583,2022-03-17T13:15:53+00:00,NaN,NaN
1369,1370,gtfs-rt,sa,CA,Québec,NaN,Exo Le Richelain,NaN,NaN,746.0,...,token,NaN,https://exo.quebec/fr/a-propos/donnees-ouverte...,45.351194,45.548627,-73.565996,-73.387583,2022-03-17T13:15:53+00:00,NaN,NaN
1370,1371,gtfs-rt,vp,CA,Québec,NaN,Exo Le Richelain,NaN,NaN,746.0,...,token,NaN,https://exo.quebec/fr/a-propos/donnees-ouverte...,45.351194,45.548627,-73.565996,-73.387583,2022-03-17T13:15:53+00:00,NaN,NaN
1470,1471,gtfs-rt,vp,CA,Québec,La Pêche,Transcollines,NaN,NaN,739.0,...,NaN,NaN,https://github.com/transcollines/GTFS-RT/blob/...,45.407263,45.919647,-77.076473,-75.617139,2022-03-17T13:15:38+00:00,NaN,NaN


In [120]:
df[df['location.bounding_box.minimum_latitude'].isnull()].iloc[0]['urls.latest']

'https://storage.googleapis.com/storage/v1/b/mdb-latest/o/ar-buenos-aires-subterraneos-de-buenos-aires-subte-gtfs-6.zip?alt=media'

In [21]:
%load_ext snakeviz

In [22]:
%%snakeviz
feeds_frequencies = []
i=0
print(i)
feed_s = feeds_t[i].copy()
feed_s.group_services()

feed_s.build_stop_clusters(distance_threshold=50)
feed_s.build_patterns(on='cluster_id')

feed_frequencies = feed_s.convert_to_frequencies(time_range=time_range)
shapes = feed_frequencies.shapes is not None
feed_frequencies.build_links_and_nodes(log=False, 
                                       shape_dist_traveled=True, 
                                       from_shape=True, 
                                       stick_nodes_on_links=True,
                                       keep_origin_columns=['departure_time','pickup_type'],
                                       keep_destination_columns=['arrival_time','drop_off_type'])
feeds_frequencies.append(feed_frequencies)

0
 
*** Profile stats marshalled to file '/tmp/tmp48ecllpr'.
Embedding SnakeViz in this document...
<function display at 0x7f2379ec88b0>


In [276]:
import gtfs_kit as gk

self = feed_frequencies.copy()

In [277]:
log=False, 
shape_dist_traveled=True, 
from_shape=True, 
stick_nodes_on_links=True,
keep_origin_columns=['departure_time','pickup_type'],
keep_destination_columns=['arrival_time','drop_off_type']

In [278]:
self.nodes

,stop_id,stop_code,stop_name,location_type,stop_timezone,parent_station,wheelchair_boarding,cluster_id,geometry
1,344,ALEX,Alexandria,0,America/Toronto,NaN,1.0,127,POINT (528241.801 5018345.855)
2,345,YONK,Yonkers,0,America/New_York,NaN,NaN,77,POINT (592391.908 4532184.963)
3,346,NIAG,Niagara Falls,0,America/New_York,NaN,1.0,152,POINT (169371.056 4780931.125)
6,341,SARN,Sarnia,0,America/Toronto,NaN,1.0,78,POINT (-102795.869 4782613.495)
8,343,SLAM,Saint-Lambert,0,America/Toronto,NaN,NaN,130,POINT (616609.082 5039477.607)
...,...,...,...,...,...,...,...,...,...
378,49,WYMT,Weymont,0,America/Toronto,NaN,NaN,7,POINT (591530.052 5305676.690)
379,46,LMAL,Lac Malouin,0,America/Toronto,NaN,NaN,4,POINT (709457.417 5252728.112)
380,47,SMQU,Summit,0,America/Toronto,NaN,NaN,3,POINT (710821.586 5296616.964)
382,45,CHER,Cherokee,0,America/Toronto,NaN,NaN,1,POINT (707730.702 5289357.040)


In [279]:

from_point='a'
to_point='b'
max_candidates=10
log=False
to_concat = []

point_dict = self.nodes.set_index('stop_id')['geometry'].to_dict()
shape_dict = self.shapes.set_index('shape_id')['geometry'].to_dict()

stop_ids = set(self.links[[from_point, to_point]].values.flatten())
stop_pts = pd.DataFrame([point_dict.get(n) for n in stop_ids], index=list(stop_ids), columns=['geometry'])

In [280]:
import geopandas as gpd
import gtfs_kit as gk
import pandas as pd
import numpy as np
from shapely.geometry import LineString, Point
from syspy.transitfeed import feed_links
from syspy.spatial import spatial
from quetzal.engine.pathfinder_utils import paths_from_edges
def cut(line, distance):
    # Cuts a line in two at a distance from its starting point
    if distance <= 0.0:
        return [None, LineString(line)]
    if distance >= line.length:
        return [LineString(line), None]
    coords = list(line.coords)
    pd = 0
    for i, p in enumerate(coords):
        if i == 0:
            continue
        pd += euclidean_distance(p, coords[i - 1])
        if pd == distance:
            return [
                LineString(coords[:i + 1]),
                LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:])]

def euclidean_distance(p1, p2):
    return np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

In [281]:
trip='628-226_0'#
trip='AC_0'
links = self.links[self.links['trip_id'] == trip].copy()
links = links.drop_duplicates(subset=['link_sequence']).sort_values(by='link_sequence')
s = shape_dict.get(links.iloc[0]['shape_id'])
trip_stops = list(links[from_point]) + [links.iloc[-1][to_point]]

# Find segments candidates from shape for projection
segments = pd.DataFrame(list(map(LineString, zip(s.coords[:-1], s.coords[1:]))), columns=['geometry'])
n_candidates = min([len(segments), max_candidates])
ng = spatial.nearest(stop_pts.loc[set(trip_stops)], segments, n_neighbors=n_candidates)
ng = ng.set_index(['ix_one', 'rank'])['ix_many'].to_dict()

# Distance matrix (stops * n_candidates)
distances_a = np.empty((len(trip_stops), n_candidates))
for r in range(n_candidates):
    proj_pts = []
    for n in trip_stops:
        seg = segments.loc[ng[(n, r)]]['geometry']
        proj_pts.append(seg.interpolate(seg.project(point_dict.get(n))))
    distances = [s.project(pts) for pts in proj_pts]
    distances_a[:, r] = distances
# TODO (faster): create custom nearest to build distance matrix directly

# Differential distance matrix (transition cost between candidates)
df = pd.DataFrame(distances_a)
diff_df = pd.DataFrame({from_point: df.index[:-1], to_point: df.index[1:]})
for i in range(n_candidates):
    for j in range(n_candidates):
        diff_col = df.iloc[:, j].shift(-1) - df.iloc[:, i]
        diff_df[(i, j)] = list(diff_col)[:-1]

diff_df = diff_df.set_index([from_point, to_point])
diff_df.columns = pd.MultiIndex.from_tuples(diff_df.columns, names=['from', 'to'])
diff_df[diff_df <= 0.0] = 1e9

# Compute diffrential between shape_dist_traveled and transition matrix
shape_dist_traveled_diff = list(self.stop_times[self.stop_times['trip_id'] == trip].sort_values(by='stop_sequence')['shape_dist_traveled'].diff())[1:]
for c in diff_df.columns:
    diff_df[c] = abs(shape_dist_traveled_diff - diff_df[c])
'''
diff_df = diff_df.stack().stack().reset_index()
diff_df['from'] = diff_df[from_point].astype(str) + '_' + diff_df['from'].astype(str)
diff_df['to'] = diff_df[to_point].astype(str) + '_' + diff_df['to'].astype(str)

# Build edges + dijkstra
candidates_e = diff_df[['from', 'to', 0]].values
source_e = np.array([["source"] * n_candidates,
                    [str(0) + '_' + str(c) for c in range(n_candidates)],
                    np.ones(n_candidates)], dtype="O").T
target_e = np.array([[str(len(trip_stops) - 1) + '_' + str(c) for c in range(n_candidates)],
                    ["target"] * n_candidates,
                    np.ones(n_candidates)], dtype="O").T

edges = np.concatenate([candidates_e, source_e, target_e])
path = paths_from_edges(edges=edges, od_set={('source', 'target')})

'''

'\ndiff_df = diff_df.stack().stack().reset_index()\ndiff_df[\'from\'] = diff_df[from_point].astype(str) + \'_\' + diff_df[\'from\'].astype(str)\ndiff_df[\'to\'] = diff_df[to_point].astype(str) + \'_\' + diff_df[\'to\'].astype(str)\n\n# Build edges + dijkstra\ncandidates_e = diff_df[[\'from\', \'to\', 0]].values\nsource_e = np.array([["source"] * n_candidates,\n                    [str(0) + \'_\' + str(c) for c in range(n_candidates)],\n                    np.ones(n_candidates)], dtype="O").T\ntarget_e = np.array([[str(len(trip_stops) - 1) + \'_\' + str(c) for c in range(n_candidates)],\n                    ["target"] * n_candidates,\n                    np.ones(n_candidates)], dtype="O").T\n\nedges = np.concatenate([candidates_e, source_e, target_e])\npath = paths_from_edges(edges=edges, od_set={(\'source\', \'target\')})\n\n'

In [282]:
self.stop_times[self.stop_times['trip_id'] == trip].sort_values(by='stop_sequence')


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
295,AC_0,25200.0,25200.0,332,1,Aéroport Montréal Pierre-Elliott Trudeau,0.0,0.0,0.0,NaN
296,AC_0,25800.0,25800.0,555,2,Dorval,0.0,0.0,NaN,NaN
297,AC_0,26700.0,26700.0,332,3,Aéroport Montréal Pierre-Elliott Trudeau,0.0,0.0,NaN,NaN


In [246]:
shape_dist_traveled_diff

[6108.599999999999, 47130.8, 46685.7, 150131.40000000002, 19791.49999999997]

In [244]:
diff_df

from           0                                                      \
to             0            1            2            3            4   
a b                                                                    
0 1     7.298347   893.129543  1267.782360     5.918881  2916.491336   
1 2   107.466338  4690.152162  4951.289956  8227.727341  8446.647946   
2 3   152.965523    71.596690   241.569497   274.878099   361.296089   
3 4    89.987263     2.722573    75.025792   158.502638   231.963322   
4 5   112.301611   120.300613   515.924292  1755.064043  2369.882520   

from                                                                    ...  \
to              5            6            7            8             9  ...   
a b                                                                     ...   
0 1   3762.940500  1675.532591  5025.101055  5098.270465   5025.101055  ...   
1 2   8642.720477  8835.942800  9026.959204   107.281205  10227.122719  ...   
2 3    844.389839  1217.671686  1726.873873  2091.628780   2213.303192  ...   
3 4    334.258260   163.696582  1661.448043  2456.500281   2075.564401  ...   
4 5   2438.590901  2515.571459  2623.337181  2730.295145   2789.926448  ...   

from            9                                                         \
to              0             1            2             3             4   
a b                                                                        
0 1   1960.872483   1075.041286   700.388470   1962.251949   4884.662166   
1 2   5139.865740   9722.551564  9983.689358  13260.126743  13479.047349   
2 3   9966.690859  10048.059691  9878.086885  10394.534481   9758.360292   
3 4   1970.350406   2057.615096  2135.363461   2218.840307   2292.300991   
4 5   2277.853275   2045.251051  1649.627372    410.487621    204.330856   

from                                                                        
to               5             6             7             8             9  
a b                                                                         
0 1    5731.111330    292.638238  9.999939e+08  9.999939e+08  9.999939e+08  
1 2   13675.119879  13868.342202  1.405936e+04  5.139681e+03  1.525952e+04  
2 3    9275.266543   8901.984695  8.392783e+03  8.028028e+03  7.906353e+03  
3 4    2394.595929   1896.641087  3.721786e+03  3.961626e+02  4.135902e+03  
4 5     273.039237    350.019795  4.577855e+02  5.647435e+02  6.243748e+02  

[5 rows x 100 columns]

In [190]:
for trip in set(self.links['trip_id']):
    print(trip)
    links = self.links[self.links['trip_id'] == trip].copy()
    links = links.drop_duplicates(subset=['link_sequence']).sort_values(by='link_sequence')
    s = shape_dict.get(links.iloc[0]['shape_id'])
    trip_stops = list(links[from_point]) + [links.iloc[-1][to_point]]

    # Find segments candidates from shape for projection
    segments = pd.DataFrame(list(map(LineString, zip(s.coords[:-1], s.coords[1:]))), columns=['geometry'])
    n_candidates = min([len(segments), max_candidates])

    ng = spatial.nearest(stop_pts.loc[set(trip_stops)], segments, n_neighbors=n_candidates)
    ng = ng.set_index(['ix_one', 'rank'])['ix_many'].to_dict()

    # Distance matrix (stops * n_candidates)
    distances_a = np.empty((len(trip_stops), n_candidates))
    for r in range(n_candidates):
        proj_pts = []
        for n in trip_stops:
            seg = segments.loc[ng[(n, r)]]['geometry']
            proj_pts.append(seg.interpolate(seg.project(point_dict.get(n))))
        distances = [s.project(pts) for pts in proj_pts]
        distances_a[:, r] = distances
    # TODO (faster): create custom nearest to build distance matrix directly

    # Differential distance matrix (transition cost between candidates)
    df = pd.DataFrame(distances_a)
    diff_df = pd.DataFrame({from_point: df.index[:-1], to_point: df.index[1:]})
    for i in range(n_candidates):
        for j in range(n_candidates):
            diff_col = df.iloc[:, j].shift(-1) - df.iloc[:, i]
            diff_df[(i, j)] = list(diff_col)[:-1]

    diff_df = diff_df.set_index([from_point, to_point])
    diff_df.columns = pd.MultiIndex.from_tuples(diff_df.columns, names=['from', 'to'])
    diff_df[diff_df <= 0.0] = 1e9

    # Compute diffrential between shape_dist_traveled and transition matrix
    shape_dist_traveled_diff = list(self.stop_times[self.stop_times['trip_id'] == trip].sort_values(by='stop_sequence')['shape_dist_traveled'].diff())[1:]
    for c in diff_df.columns:
        diff_df[c] = abs(shape_dist_traveled_diff - diff_df[c])

    diff_df = diff_df.stack().stack().reset_index()
    diff_df['from'] = diff_df[from_point].astype(str) + '_' + diff_df['from'].astype(str)
    diff_df['to'] = diff_df[to_point].astype(str) + '_' + diff_df['to'].astype(str)

    # Build edges + dijkstra
    candidates_e = diff_df[['from', 'to', 0]].values
    source_e = np.array([["source"] * n_candidates,
                        [str(0) + '_' + str(c) for c in range(n_candidates)],
                        np.ones(n_candidates)], dtype="O").T
    target_e = np.array([[str(len(trip_stops) - 1) + '_' + str(c) for c in range(n_candidates)],
                        ["target"] * n_candidates,
                        np.ones(n_candidates)], dtype="O").T

    edges = np.concatenate([candidates_e, source_e, target_e])
    path = paths_from_edges(edges=edges, od_set={('source', 'target')})
    best_distances = [int(p.split('_')[-1]) for p in path['path'][0][1:-1]]

    distances = list(np.take_along_axis(distances_a, np.array(best_distances)[:, np.newaxis], axis=1).flatten())

    # Cut shape
    cuts = []
    for d1, d2 in zip(distances[:-1], distances[1:]):
        first_cut = cut(s, d1)[1]
        cuts.append(cut(first_cut, d2 - d1)[0])

    for i, c in enumerate(cuts):
        if c is None:
            msg = f'''
            Failed to cut shape for trip {trip}. Replacing by A -> B Linestring.
            '''
            if log:
                print(msg)
            cuts = linestring_geometry(links, point_dict, 'a', 'b').values
    if abs(1 - sum([c.length for c in cuts]) / s.length) > 0.1:
        msg = f'''
            Length of trip {trip} is more than 10% longer or shorter than shape.
            This may be caused by a bad cutting of the shape.
            Try increasing the maximum number of candidates (max_candidates)
            '''
        if log:
            print(msg)

    links['geometry'] = cuts

    to_concat.append(links)

628-226_0
AC_0


KeyError: 0